In [144]:
import torch
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


In [145]:
#exploration du dataset

df = pd.read_csv('../dataset.csv')

df.head(1)


,Unnamed: 0,url,title,description,cat1,cat2,cat3
0,0,https://www.tourisme-cambresis.fr/1-les-templiers-coffre.html,"Aventure-jeu : ""Les Templiers du coffre d'or""","Le jeu aventure « Les templiers du coffre d’or » créé par la ville de Caudry en partenariat avec l’association Caudry Ma Passion débarque !\n\nConstruit autour d’un scénario original et vous entraînant dans un jeu de piste en autonomie dans le centre-ville de Caudry, ce jeu aventure vous permettra de passer 2h de divertissement en famille.\n\nVous pourrez vous procurer le sac contenant tout le nécessaire pour vous lancer dans cette aventure pour la somme de 15€ (un sac permet à une famille entière de jouer) auprès du Bureau d'Information Touristique de Caudry.",Jeu,Famille,Détente


In [146]:
#garder colonne description, cat1, cat2, cat3

df = df[['description', 'cat1', 'cat2', 'cat3']]

In [147]:
#supprimer les retours à la ligne dans la colonne description

df['description'] = df['description'].str.replace('\n', ' ')
df.head(1)


,description,cat1,cat2,cat3
0,"Le jeu aventure « Les templiers du coffre d’or » créé par la ville de Caudry en partenariat avec l’association Caudry Ma Passion débarque ! Construit autour d’un scénario original et vous entraînant dans un jeu de piste en autonomie dans le centre-ville de Caudry, ce jeu aventure vous permettra de passer 2h de divertissement en famille. Vous pourrez vous procurer le sac contenant tout le nécessaire pour vous lancer dans cette aventure pour la somme de 15€ (un sac permet à une famille entière de jouer) auprès du Bureau d'Information Touristique de Caudry.",Jeu,Famille,Détente


In [148]:
#remplacer les valeurs manquantes de cat1, cat2, cat3 par 'neant'

df['cat1'] = df['cat1'].fillna('neant')
df['cat2'] = df['cat2'].fillna('neant')
df['cat3'] = df['cat3'].fillna('neant')

df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 390 entries, 0 to 390
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  390 non-null    object
 1   cat1         390 non-null    object
 2   cat2         390 non-null    object
 3   cat3         390 non-null    object
dtypes: object(4)
memory usage: 15.2+ KB


In [149]:
#encodage cat1 cat2 cat3

df_encoded = pd.get_dummies(df, columns=['cat1', 'cat2', 'cat3'])

df_encoded.head(1)

,description,cat1_Action,cat1_Atelier,cat1_Balade,cat1_Brocante,cat1_Concert,cat1_Conférence,cat1_Culture,cat1_Danse,cat1_Détente,cat1_Environnement,cat1_Exposition,cat1_Famille,cat1_Festival,cat1_Fête,cat1_Jeu,cat1_Marché,cat1_Spectacle,cat1_Sport,cat1_Théatre,cat1_Visite,cat2_Action,cat2_Art,cat2_Atelier,cat2_Balade,cat2_Brocante,cat2_Concert,cat2_Culture,cat2_Danse,cat2_Détente,cat2_Environnement,cat2_Exposition,cat2_Famille,cat2_Festival,cat2_Fête,cat2_Gastronomie,cat2_Histoire,cat2_Jeu,cat2_Marché,cat2_Santé,cat2_Spectacle,cat2_Sport,cat2_Théatre,cat2_Visite,cat2_neant,cat3_Art,cat3_Atelier,cat3_Balade,cat3_Concert,cat3_Culture,cat3_Détente,cat3_Environnement,cat3_Exposition,cat3_Famille,cat3_Festival,cat3_Fête,cat3_Gastronomie,cat3_Histoire,cat3_Jeu,cat3_Marché,cat3_Santé,cat3_Spectacle,cat3_Sport,cat3_Théatre,cat3_neant
0,"Le jeu aventure « Les templiers du coffre d’or » créé par la ville de Caudry en partenariat avec l’association Caudry Ma Passion débarque ! Construit autour d’un scénario original et vous entraînant dans un jeu de piste en autonomie dans le centre-ville de Caudry, ce jeu aventure vous permettra de passer 2h de divertissement en famille. Vous pourrez vous procurer le sac contenant tout le nécessaire pour vous lancer dans cette aventure pour la somme de 15€ (un sac permet à une famille entière de jouer) auprès du Bureau d'Information Touristique de Caudry.",False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [150]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 390 entries, 0 to 390
Data columns (total 65 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   description         390 non-null    object
 1   cat1_Action         390 non-null    bool  
 2   cat1_Atelier        390 non-null    bool  
 3   cat1_Balade         390 non-null    bool  
 4   cat1_Brocante       390 non-null    bool  
 5   cat1_Concert        390 non-null    bool  
 6   cat1_Conférence     390 non-null    bool  
 7   cat1_Culture        390 non-null    bool  
 8   cat1_Danse          390 non-null    bool  
 9   cat1_Détente        390 non-null    bool  
 10  cat1_Environnement  390 non-null    bool  
 11  cat1_Exposition     390 non-null    bool  
 12  cat1_Famille        390 non-null    bool  
 13  cat1_Festival       390 non-null    bool  
 14  cat1_Fête           390 non-null    bool  
 15  cat1_Jeu            390 non-null    bool  
 16  cat1_Marché         390 non-nul

In [151]:
#initialise le tokenizer et le modèle

tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096")

#préapration des données
description = df_encoded['description'].tolist()
inputs = tokenizer(description, return_tensors="pt", padding=True, truncation=True, max_length=4096)

#labels

labels = df_encoded.drop(columns='description').values
labels = torch.tensor(labels)

#passer les données dans le modèle

outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits




/home/utilisateur/Documents/dev_ia/fine-tune-classification/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Initializing global attention on CLS token...


: 